In [1]:

from keras.preprocessing import sequence
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from keras.utils import np_utils
from sklearn import preprocessing




/Users/erin/miniconda3/envs/project764/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import math

train_gs_file = './data764/NEEL2016-training_neel.gs'
train_tsv_file = './data764/NEEL2016-training.tsv'
test_gs_file = './data764/NEEL2016-test_neel.gs'
test_tsv_file = './data764/NEEL2016-test.tsv'

#col_names=['tweet_id','start','end','uri', 'confidence', 'type']

with open(train_gs_file) as f:
    train_gs_data = f.readlines()
with open(train_tsv_file) as f:
    train_tsv_data = f.readlines()
with open(test_gs_file) as f:
    test_gs_data = f.readlines()
with open(test_tsv_file) as f:
    test_tsv_data = f.readlines()
train_gs_data[0].split()
train_tweet_dict = {}
for line in train_tsv_data:
    data = line.split('|,|')
    if (len(data) == 2):
        tweet_id = data[0][1:]
        tweet_text = data[1].strip()[:-1]
        if (tweet_id is not None and tweet_text is not None):
            train_tweet_dict[tweet_id] = tweet_text
train_word = []
train_entity = []
for line in train_gs_data:
    data = line.split()
    if (len(data) == 6):
        tweet_id = data[0].strip()
        start_index = int(data[1].strip())
        end_index = int(data[2].strip())
        entity = data[5].strip()
        
        if (entity == 'Organization373937812812615000'):
            entity = 'ORG'
        elif (entity == 'Location'):
            entity = 'LOC'
        elif (entity == 'Person'):
            entity = 'PER'
        elif (entity == 'Organization'):
            entity = 'ORG'
        else:
            entity = 'O'   
        train_entity.append(entity)
        
        word = train_tweet_dict[tweet_id][start_index:end_index]
        train_word.append(word)
        
        
        
        
        test_tweet_dict = {}
for line in test_tsv_data:
    data = line.split('|,|')
    if (len(data) == 2):
        tweet_id = data[0][1:]
        tweet_text = data[1].strip()[:-1]
        if (tweet_id is not None and tweet_text is not None):
            test_tweet_dict[tweet_id] = tweet_text
            
test_word = []
test_entity = []
for line in test_gs_data:
    data = line.split()
    if (len(data) == 6):
        tweet_id = data[0].strip()
        start_index = int(data[1].strip())
        end_index = int(data[2].strip())
        entity = data[5].strip()
        
        if (entity == 'Organization373937812812615000'):
            entity = 'ORG'
        elif (entity == 'Location'):
            entity = 'LOC'
        elif (entity == 'Person'):
            entity = 'PER'
        elif (entity == 'Organization'):
            entity = 'ORG'
        else:
            entity = 'O'  
        test_entity.append(entity)
        
        word = test_tweet_dict[tweet_id][start_index:end_index]
        test_word.append(word)
        
        
        
number_of_labels = set(train_entity+test_entity)




all_chars = []
for word in train_word:
    all_chars.append(word)
for word in test_word:
    all_chars.append(word)  


unique_chars = set(all_chars)
#unique_chars

word_dict = {}
for i, char in enumerate(unique_chars):
    #print(f'i:{i}, char:{char}')
    word_dict[char] = i + 1 # + 1 if you want to mask


train_words_numeric = []
test_words_numeric = []
for word in train_word:
    train_words_numeric.append(word_dict[word])
for word in test_word:
    test_words_numeric.append(word_dict[word])

train_feature = np.expand_dims(train_words_numeric, axis=2)
test_feature = np.expand_dims(test_words_numeric, axis=2)
train_feature.shape

/Users/erin/miniconda3/envs/project764/lib/python3.6/site-packages/ipykernel_launcher.py:119: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
/Users/erin/miniconda3/envs/project764/lib/python3.6/site-packages/ipykernel_launcher.py:120: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


(8665, 1)

In [3]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(train_feature, train_entity).predict(test_feature)

right = [0] *4
wrong = [0] *4

for x in range(0, len(test_entity)):
    if y_pred[x] == test_entity[x]:
        if y_pred[x] == 'LOC':
            right[0] = right[0] + 1
        elif y_pred[x] == 'PER':
            right[1] = right[1] + 1
        elif y_pred[x] == 'ORG':
            right[2] = right[2] + 1
        else:
            right[3] = right[3] + 1
    else:
        if y_pred[x] == 'LOC':
            wrong[0] = wrong[0] + 1
        elif y_pred[x] == 'PER':
            wrong[1] = wrong[1] + 1
        elif y_pred[x] == 'ORG':
            wrong[2] = wrong[2] + 1
        else:
            wrong[3] = wrong[3] + 1
        

for x in range(0, len(right)):
    if right[x]+wrong[x] > 0:
        print("Accuracy (", right[x], "/", right[x]+wrong[x], "): ", right[x]/(right[x]+wrong[x]))
    else:
        print("Accuracy (0/0): 0")


print("Number of mislabeled points out of a total %d points : %d. %f"
 % (test_feature.shape[0],(test_entity != y_pred).sum(), ((test_entity != y_pred).sum() / test_feature.shape[0])))

Accuracy (0/0): 0
Accuracy ( 331 / 1005 ):  0.32935323383084575
Accuracy (0/0): 0
Accuracy ( 5 / 17 ):  0.29411764705882354
Number of mislabeled points out of a total 1022 points : 686. 0.671233
